# The Modern Tool Calling Way of LangChain

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

tools = [multiply, exponentiate, add]


llm = ChatOpenAI(model="gpt-4o", temperature=0)


agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241", })



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 3, 'y': 5}`


8.0
Invoking: `exponentiate` with `{'x': 5, 'y': 2.743}`


82.65606421491815
Invoking: `add` with `{'x': 17.24, 'y': -918.1241}`


-900.8841Here are the results:

1. \( 3 + 5^{2.743} \) is approximately \( 3 + 82.656 \approx 85.656 \).
2. \( 17.24 - 918.1241 \) is approximately \( -900.8841 \).

> Finished chain.


{'input': "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241",
 'output': 'Here are the results:\n\n1. \\( 3 + 5^{2.743} \\) is approximately \\( 3 + 82.656 \\approx 85.656 \\).\n2. \\( 17.24 - 918.1241 \\) is approximately \\( -900.8841 \\).'}

source: https://arc.net/l/quote/cdomnqvm

So when should you use with_structured_output versus binding tools and reading tool calls directly?

with_structured_output always returns a structured output in the schema that you specified. This is useful when you want to force the LLM to output information that matches a specific schema. This is useful for information extraction tasks.

bind_tools is more general and can select a specific tool - or no tool, or multiple tools! This is useful when you want to allow the LLM to have more flexibility in how it should respond - for example, in agent applications where you need to choose which tools to invoke but also respond to the user.